In [1]:
import rlcard
import torch
import pathlib
import numpy
import ray
import models
import ddz

import self_play_test
import time
from rlcard.agents import RandomAgent
from rlcard.utils import set_seed, tournament

In [2]:
config = ddz.MuZeroConfig()
observation_shapes = config.observation_shapes
configs = []
for i in range(config.num_players):
    config = ddz.MuZeroConfig()
    config.observation_shape = observation_shapes[i]
    configs.append(config)

In [3]:
agents = []
for i in range(config.num_players):
    model = models.MuZeroNetwork(configs[i])
    model = model.cuda()
    model.eval()
    agents.append(model)

ray.init(num_gpus=1.0, ignore_reinit_error=True)

2022-10-13 11:52:48,332	INFO worker.py:1515 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


Python version:,3.7.13
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8266


In [4]:
landlord_paths = '/home/yg/code/Muzero//muzero/results/test_ddz/new/model0.checkpoint'
farmer0_paths = '/home/yg/code/Muzero/muzero/results/test_ddz/new/model1.checkpoint'
farmer1_paths = '/home/yg/code/Muzero/muzero/results/test_ddz/new/model2.checkpoint'

models_paths = [landlord_paths,farmer0_paths,farmer1_paths]
# map_location = torch.device('cpu')
for i in range(3):
    model_path = pathlib.Path(models_paths[i])
    model_checkpoint = torch.load(model_path)
    agents[i].set_weights(model_checkpoint['weights'])

In [5]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     numpy.random.seed(seed)
     torch.backends.cudnn.deterministic = True

In [6]:
seed = 42
setup_seed(seed)
num_test = 100
env = ddz.Game(seed)
env.set_agents(agents)

set_seed------------


In [ ]:
self_play_worker = self_play_test.SelfPlayTest.options(
                            num_cpus=0,
                            num_gpus=1.0,
                        ).remote(env, config, configs, seed, agents)

results = []
for i in range(num_test):
    start_time = time.time()
    results.append(
        ray.get(
            self_play_worker.play_game.remote(
                0,
                0,
                False,
                "self",
                0,
            )
        )
    )
    end_time = time.time()
    print('time',end_time-start_time, i)

time 53.270206689834595 0
(SelfPlayTest pid=24815) {'hand_cards_with_suit': ['', 'H4 S4 C6 H6 S7 DT CT ST CJ SJ CQ BJ', 'C3 D3 C4 D5 S6 C7 H7 H8 C9 S9 HT DJ CK HK DA C2 D2'], 'hand_cards': ['', '44667TTTJJQB', '3345677899TJKKA22'], 'trace': [(0, 'AAA2'), (1, 'pass'), (2, 'pass'), (0, '5888'), (1, 'pass'), (2, 'pass'), (0, '55'), (1, 'QQ'), (2, 'pass'), (0, 'pass'), (1, '3'), (2, 'pass'), (0, 'R'), (1, 'pass'), (2, 'pass'), (0, 'J'), (1, 'K'), (2, 'pass'), (0, 'pass'), (1, '6'), (2, 'pass'), (0, 'K'), (1, 'pass'), (2, 'pass'), (0, '4'), (1, 'pass'), (2, 'pass'), (0, '3'), (1, 'pass'), (2, 'pass'), (0, '99'), (1, 'pass'), (2, 'pass'), (0, 'Q'), (1, 'pass'), (2, 'pass'), (0, '2'), (1, 'pass'), (2, 'pass'), (0, '7')], 'current_player': 1, 'legal_actions': []}
(SelfPlayTest pid=24815) [1 0 0]
time 40.60608768463135 1
(SelfPlayTest pid=24815) {'hand_cards_with_suit': ['', 'D3 C4 H5 C5 S5 C6 S7 C7 C9 HT SQ HK SK DA SA D2', 'C3 S3 H4 S4 D5 S6 H7 S8 D8 DT SJ CK DK CA HA'], 'hand_cards': ['', '3

In [ ]:
res_last = []
result = [[],[],[]]
for game_historys in results:
    for i in range(3):
        result[i].append(
            sum(
                reward
                for k, reward in enumerate(game_historys[i].reward_history)
            )
        )

In [ ]:
for i in range(3):
    res_last.append(numpy.mean(result[i]))

print(res_last)